<a href="https://colab.research.google.com/github/dmswl0707/content_based_filtering_RecomendationSystem/blob/main/01_content_based_filtering_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/RCS_data/01data.csv')
data.head(2)

In [ ]:
del data['Unnamed: 0']

In [ ]:
df = data
df.head(2)
df.info()

In [ ]:
###최종으로 사용할 데이터 (이전 메타데이터에서 실습데이터로 전처리)
df = df[['id','genres', 'vote_average', 'vote_count','popularity','title',  'keywords', 'overview']]
df.copy()

# 평가 기준 조정 
**영화 평점 vote_average에 대한 이슈**
투표 결과가 많을 수록 평점이 낮아지는 이슈에 대한 해결 

**( v / (v+m) * R ) + (m / (m + v) * C)** 을 이용한다.

- r : 개별 영화 평점
- v : 개별 영화에 평점을 투표한 횟수
- m : 500위 안에 들어야 하는 최소 투표
- c : 전체 영화에 대한 평균 평점


**영화 후보군을 정하기 위한 상위선 기준**
- 탑 500위 영화들은 상위 몇 퍼센트 영화인가? 답은 89%
(상위 88%는 529회이기 때문에 89퍼센트으로 가정한다)

#데이터 집계하기(C = column)
- df.describe() = 컬럼 value에 대한 info
- df['C'].quantile(퍼센트) = 백분율 값 리턴
- df.grouby(['C']).count() = 건수
- df.grouby(['C']).mean() = 평균
- df.grouby(['C']).sum() = 합계
- df.grouby(['C']).std() = 표준편차


---
###더 알아보기
df.groupby(['C']) : 중복된 row 값으로 그룹별 value 리턴
df.groupby(['지역'])['판매'].transform('count/mean/sum/std') = 지역별 판매 (집계 종류)



In [ ]:
# 백분율 88% 득표 값
tmp_m = df['vote_count'].quantile(0.88)
tmp_m 

In [ ]:
#df에서 투표값이 1410.8보다 같거나 크다면 tmp_data로 정의한다
tmp_data = df.copy().loc[df['vote_count'] >= tmp_m]
print(tmp_data)
tmp_data.shape #4402 중에서 529개가 있다

In [ ]:
#상위 89퍼센트 총 485위 데이터
m = df['vote_count'].quantile(0.89)
data = df.loc[df['vote_count'] >= m]
data.head()
data.info()

C = data['vote_average'].mean()
print(C)
print(m)

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return ( v / (v+m) * R ) + (m / (m + v) * C)

In [ ]:
data['score'] = data.apply(weighted_rating, axis = 1)

data.head(5)
data.info()